In [64]:
import pandas as pd
import tensorflow_hub as hub
import openai 
import numpy as np
import altair as alt
from openai import OpenAI
from sql_metadata import Parser
import os
from IPython.display import display, Markdown
import ast
import cx_Oracle
import re

In [3]:
# Define elements to connect to datawarehouse
username = ''
password = '' 
dsn = 'dwprod.mit.edu:1521/dwrhs.mit.edu'  # Data Source Name (TNS entry name)
api_key =''

# Read the MIT DW queries 

In [21]:
df_q = pd.read_json('/home/skikk/Dokumente/uni/WS2324/MIT/llm-enterprise/data/50_queries_with_ambiguity.json')

In [22]:
df_q

,question,sql
0,How many organisation units in human resources...,[SELECT COUNT(*) FROM (SELECT hr_org_unit_key ...
1,How many organisation units in human resources...,[SELECT COUNT(*) FROM (SELECT hr_org_unit_key ...
2,COUNT the number of courses offered by each de...,"[SELECT DEPARTMENT_CODE, COUNT(*) AS course_CO..."
3,"What are the past, upcoming, and current courses?","[SELECT DISTINCT subject_title, effective_term..."
4,Which faculty has had the most location changes?,[SELECT MAX(cnt) FROM (SELECT COUNT(*) AS cnt ...
5,What is the average number of courses a studen...,[SELECT AVG(cnt) FROM (SELECT COUNT(*) AS cnt ...
6,Which students are employed by mit?,[SELECT DISTINCT empl.full_name FROM mit_stude...
7,What is the shortest course available this sem...,"[SELECT DISTINCT subject_title, term_end_date ..."
8,Show buildings and their address.,"[SELECT fb.building_name, fba.* FROM wareuser...."
9,How many students do we have?,[SELECT COUNT(*) FROM mit_student_directory;]


## Identify the tables used by each query 

In [27]:
questions = []
tables = []
sqls=[]
for i in range(len(df_q)):
    sql = []
    table = []
    for q in df_q['sql'][i]:
        q = q.upper().replace('WAREUSER.', '')
        t = Parser(q).tables
        for v in t:
            table.append(v.upper())
        sql.append(q.upper())
    questions.append(df_q['question'][i])
    sqls.append(sql)
    tables.append(table)
df = pd.DataFrame(data={"question":questions, "sql":sqls, "tables":tables})

In [28]:
df

,question,sql,tables
0,How many organisation units in human resources...,[SELECT COUNT(*) FROM (SELECT HR_ORG_UNIT_KEY ...,"[HR_ORG_UNIT_NEW, HR_ORG_UNIT]"
1,How many organisation units in human resources...,[SELECT COUNT(*) FROM (SELECT HR_ORG_UNIT_KEY ...,"[HR_ORG_UNIT_NEW, HR_ORG_UNIT]"
2,COUNT the number of courses offered by each de...,"[SELECT DEPARTMENT_CODE, COUNT(*) AS COURSE_CO...","[CIS_COURSE_CATALOG, COURSE_CATALOG_SUBJECT_OF..."
3,"What are the past, upcoming, and current courses?","[SELECT DISTINCT SUBJECT_TITLE, EFFECTIVE_TERM...","[CIS_COURSE_CATALOG, ACADEMIC_TERMS, DRUPAL_CO..."
4,Which faculty has had the most location changes?,[SELECT MAX(CNT) FROM (SELECT COUNT(*) AS CNT ...,[FCLT_BUILDING_ADDRESS_HIST]
5,What is the average number of courses a studen...,[SELECT AVG(CNT) FROM (SELECT COUNT(*) AS CNT ...,"[MIT_STUDENT_DIRECTORY, CIS_COURSE_CATALOG, MI..."
6,Which students are employed by mit?,[SELECT DISTINCT EMPL.FULL_NAME FROM MIT_STUDE...,"[MIT_STUDENT_DIRECTORY, EMPLOYEE_DIRECTORY, MI..."
7,What is the shortest course available this sem...,"[SELECT DISTINCT SUBJECT_TITLE, TERM_END_DATE ...","[ACADEMIC_TERMS, CIS_COURSE_CATALOG, ACADEMIC_..."
8,Show buildings and their address.,"[SELECT FB.BUILDING_NAME, FBA.* FROM FAC_BUILD...","[FAC_BUILDING, FAC_BUILDING_ADDRESS]"
9,How many students do we have?,[SELECT COUNT(*) FROM MIT_STUDENT_DIRECTORY;],[MIT_STUDENT_DIRECTORY]


## Identify the unique tables referred by the corpus 

In [29]:
unique_tables = set()
for  l in tables: 
    for t in l:
        unique_tables.add(t.upper())

In [30]:
unique_tables

{'ACADEMIC_TERMS',
 'ACADEMIC_TERMS_ALL',
 'ACADEMIC_TERM_PARAMETER',
 'CIS_COURSE_CATALOG',
 'COURSE_CATALOG_SUBJECT_OFFERED',
 'DRUPAL_COURSE_CATALOG',
 'DRUPAL_EMPLOYEE_DIRECTORY',
 'EMPLOYEE_DIRECTORY',
 'FAC_BUILDING',
 'FAC_BUILDING_ADDRESS',
 'FAC_FLOOR',
 'FAC_MAJOR_USE',
 'FAC_ROOMS',
 'FCLT_BUILDING',
 'FCLT_BUILDING_ADDRESS_HIST',
 'FCLT_BUILDING_HIST',
 'FCLT_FLOOR',
 'FCLT_MAJOR_USE',
 'FCLT_ROOMS',
 'HR_ORG_UNIT',
 'HR_ORG_UNIT_NEW',
 'IAP_SUBJECT_CATEGORY',
 'IAP_SUBJECT_DETAIL',
 'IAP_SUBJECT_PERSON',
 'IAP_SUBJECT_SESSION',
 'LIBRARY_COURSE_INSTRUCTOR',
 'LIBRARY_MATERIAL_STATUS',
 'LIBRARY_RESERVE_CATALOG',
 'LIBRARY_RESERVE_MATRL_DETAIL',
 'LIBRARY_SUBJECT_OFFERED',
 'MIT_STUDENT_DIRECTORY',
 'SE_PERSON',
 'SIS_COURSE_DESCRIPTION',
 'SPACE_DETAIL',
 'STUDENT_DEPARTMENT',
 'SUBJECT_ENROLLABLE',
 'SUBJECT_OFFERED',
 'SUBJECT_OFFERED_SUMMARY',
 'SUBJECT_SELECTOR',
 'SUBJECT_SUMMARY',
 'TIP_DETAIL',
 'TIP_MATERIAL',
 'TIP_MATERIAL_STATUS',
 'TIP_SUBJECT_OFFERED',
 'TOP_L

In [31]:
len(unique_tables)

47

In [32]:
prefix='/home/skikk/Dokumente/uni/WS2324/MIT/llm-enterprise/data/views/'

## Go thrugh the `unique_tables` list and save the respective schemas as proper CSV files 

In [33]:
for t in unique_tables: 
    if t.upper().startswith('WAREUSER.'):
        t = t[len('WAREUSER.'):]
    schema_file = prefix + 'schema/' + t + '.csv'
    if os.path.isfile(schema_file): 
        s = pd.read_csv(schema_file, header=None)
        s = s.drop(1) # delete the "header" line 
        column_names = [h.strip() for h in s[0][0].split(';')]
        s_new = pd.DataFrame(list(s[0][1:].apply(lambda row: [c.strip() for c in row.split(';')])), columns=column_names)
        new_schema_file = prefix + 'schema_new/' + t + '.csv'
        s_new.to_csv(new_schema_file, index=False)

# Treat ChatGPT as an all-knowing oracle (without any additional information)
## Prompt
_Your job is to write SQL queries that answer a user's question using the tables in the MIT Data Warehouse. 
The MIT Data Warehouse is a central data source that combines data from various administrative systems at 
MIT, containing information about students, faculty, and personnel. You can find more about the MIT data 
warehouse tables at https://web.mit.edu/warehouse/metadata/tables/all_tables.html._

_How many organisation units in human resources are relabeled?_

In [34]:
client = OpenAI(api_key=api_key)

## Populate the prompt and call the Open AI API

In [ ]:
import requests
headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {api_key}'
}

response = requests.get('https://api.openai.com/v1/engines', headers=headers)

if response.status_code == 200:
    engines = response.json()['data']
    print("Available engines:")
    for engine in engines:
        print(engine['id'])
else:
    print("Error:", response.text)

In [37]:
# TODO: do this in batch 
gpt_sql_nocontext = []
for q in df['question']: 
    response = client.chat.completions.create(
        model="gpt-4",
        temperature=0,
        messages = [
          {
          "role": "system",
          "content": "Your job is to write SQL queries that answer a user's question using the tables in the MIT Data Warehouse. \
          The MIT Data Warehouse is a central data source that combines data from various administrative systems at MIT, containing \
          information about students, faculty, and personnel. You can find more about the MIT data warehouse tables at \
          https://web.mit.edu/warehouse/metadata/tables/all_tables.html. \
          Reply with as many answers \
          as you can think of in SQL and include no linebreaks, newlines, escape characters or other commentary for the each possible \
          answer, but seperate the possibilities by commas."
          # Provide your answer in JSON form."
          # Reply with only the answer in JSON form and include no other commentary"
          },
            {"role":"user", 
             "content":q}]
    )
    gpt_sql_nocontext.append(response.choices[0].message.content)

In [38]:
df['gpt_sql_nocontext']=gpt_sql_nocontext

In [39]:
df

,question,sql,tables,gpt_sql_nocontext
0,How many organisation units in human resources...,[SELECT COUNT(*) FROM (SELECT HR_ORG_UNIT_KEY ...,"[HR_ORG_UNIT_NEW, HR_ORG_UNIT]",SELECT COUNT(*) FROM hr_organization_units WHE...
1,How many organisation units in human resources...,[SELECT COUNT(*) FROM (SELECT HR_ORG_UNIT_KEY ...,"[HR_ORG_UNIT_NEW, HR_ORG_UNIT]",SELECT COUNT(*) FROM human_resources WHERE is_...
2,COUNT the number of courses offered by each de...,"[SELECT DEPARTMENT_CODE, COUNT(*) AS COURSE_CO...","[CIS_COURSE_CATALOG, COURSE_CATALOG_SUBJECT_OF...","""SELECT department_id, COUNT(*) FROM courses G..."
3,"What are the past, upcoming, and current courses?","[SELECT DISTINCT SUBJECT_TITLE, EFFECTIVE_TERM...","[CIS_COURSE_CATALOG, ACADEMIC_TERMS, DRUPAL_CO...",SELECT * FROM courses WHERE course_date < CURR...
4,Which faculty has had the most location changes?,[SELECT MAX(CNT) FROM (SELECT COUNT(*) AS CNT ...,[FCLT_BUILDING_ADDRESS_HIST],"SELECT faculty_id, COUNT(DISTINCT location_id)..."
5,What is the average number of courses a studen...,[SELECT AVG(CNT) FROM (SELECT COUNT(*) AS CNT ...,"[MIT_STUDENT_DIRECTORY, CIS_COURSE_CATALOG, MI...",SELECT AVG(course_count) FROM (SELECT student_...
6,Which students are employed by mit?,[SELECT DISTINCT EMPL.FULL_NAME FROM MIT_STUDE...,"[MIT_STUDENT_DIRECTORY, EMPLOYEE_DIRECTORY, MI...",SELECT student_id FROM students WHERE employme...
7,What is the shortest course available this sem...,"[SELECT DISTINCT SUBJECT_TITLE, TERM_END_DATE ...","[ACADEMIC_TERMS, CIS_COURSE_CATALOG, ACADEMIC_...",SELECT MIN(course_length) FROM courses WHERE s...
8,Show buildings and their address.,"[SELECT FB.BUILDING_NAME, FBA.* FROM FAC_BUILD...","[FAC_BUILDING, FAC_BUILDING_ADDRESS]","SELECT building_name, building_address FROM bu..."
9,How many students do we have?,[SELECT COUNT(*) FROM MIT_STUDENT_DIRECTORY;],[MIT_STUDENT_DIRECTORY],"SELECT COUNT(*) FROM students, SELECT COUNT(st..."


# Give ChatGPT the "database" schema as context and elicit its answer


In [40]:
def table_to_sql_create(t): 
    schema_file=prefix + 'schema_new/' + t + '.csv'
    p = None
    if os.path.isfile(schema_file):
        df_t = pd.read_csv(schema_file)
        p = "CREATE TABLE "+ t+ "(\n"  
        p = p + '\n'.join(list(df_t[['COLUMN_NAME', 'DATA_TYPE']].apply(lambda row: ' '.join(row.astype(str)), axis=1)))
        p = p + ')\n'
    return p
    

#### Generalize table-names (delete "WAREUSER." in front) and create the prefix for the prompt.


In [41]:
prompt = "** all the tables in the database ** \n\n"
cnt = 0 
for t in unique_tables:
    if t.upper().startswith('WAREUSER.'):
        t = t[len('WAREUSER.'):]
    p = table_to_sql_create(t)
    if not p: 
        print(t)
    if p: 
        cnt = cnt + 1
        prompt = prompt + p
        prompt = prompt + "\n\n"


#### There should be no tables listed above (only if the table-name does not occur in our extracted table-schemas).

In [42]:
## The "schema" prompt, which we append to the system prompt below 
print(prompt)

** all the tables in the database ** 

CREATE TABLE TOP_LEVEL_DOMAIN(
TOP_LEVEL_DOMAIN_KEY VARCHAR2
TOP_LEVEL_DOMAIN VARCHAR2
TOP_LEVEL_DOMAIN_DESCRIPTION VARCHAR2
WAREHOUSE_LOAD_DATE DATE)


CREATE TABLE FCLT_MAJOR_USE(
FCLT_MAJOR_USE_KEY VARCHAR2
MAJOR_USE VARCHAR2
DESCRIPTION VARCHAR2
ASSIGNABLE VARCHAR2
WAREHOUSE_LOAD_DATE DATE)


CREATE TABLE ACADEMIC_TERM_PARAMETER(
TERM_PARAMETER VARCHAR2
TERM_INDICATOR VARCHAR2
TERM_CODE VARCHAR2
TERM_DESCRIPTION VARCHAR2
TERM_START_DATE DATE
TERM_END_DATE DATE
TERM_LAST_DAY_BEFORE_NEXT_TERM DATE
IS_CURRENT_TERM VARCHAR2)


CREATE TABLE HR_ORG_UNIT(
HR_ORG_UNIT_KEY VARCHAR2
HR_ORG_UNIT_ID VARCHAR2
HR_ORG_UNIT_TITLE VARCHAR2
HR_ORG_UNIT_LEVEL VARCHAR2
HR_DEPARTMENT_ID VARCHAR2
HR_DEPARTMENT_CODE VARCHAR2
HR_DEPARTMENT_CODE_OLD VARCHAR2
HR_DEPARTMENT_NAME VARCHAR2
HR_DEPARTMENT_NAME_LONG VARCHAR2
HR_DEPARTMENT_NAME_ALPHA VARCHAR2
ORG_HIER_SCHOOL_AREA_NAME VARCHAR2
ORG_HIER_TOP_LEVEL_NAME VARCHAR2
ORG_HIER_ROOT_NAME VARCHAR2
HR_ORG_LEVEL1_ID VARCH

## We're now ready to make the api call 

In [43]:
gpt_sql_global_schema = []
for q in df['question']: 
    response = client.chat.completions.create(
        model="gpt-4",
        temperature = 0,
        messages = [
          {
          "role": "system",
          "content": "Your job is to write SQL queries that answer a user's question using the tables in the MIT Data Warehouse. \
          The MIT Data Warehouse is a central data source that combines data from various administrative systems at MIT, containing \
          information about students, faculty, and personnel. The schemas of all the tables that you might need \
          for answering the user questions are below. There are 50 tables in the database.  Reply with as many answers \
           as you can think of in SQL and include no linebreaks, newlines, escape characters or other commentary for the each possible \
           answer, but seperate the possibilities by semicolons. \n\n" + prompt 
          # Provide your answer in JSON form."
          # Reply with only the answer in JSON form and include no other commentary"
        },
          {"role":"user", 
           "content":q}]
          )
    gpt_sql_global_schema.append(response.choices[0].message.content)
    print(response.choices[0].message.content)
df['gpt_sql_global_schema']=gpt_sql_global_schema
df.to_csv('/home/skikk/Dokumente/uni/WS2324/MIT/llm-enterprise/data/gpt_sql_v0.csv', index=False)

SELECT COUNT(*) FROM HR_ORG_UNIT WHERE HR_ORG_UNIT_TITLE != HR_ORG_UNIT_ID, SELECT COUNT(*) FROM HR_ORG_UNIT_NEW WHERE HR_ORG_UNIT_TITLE != HR_ORG_UNIT_ID
SELECT COUNT(*) FROM HR_ORG_UNIT_NEW WHERE HR_ORG_UNIT_ID NOT IN (SELECT HR_ORG_UNIT_ID FROM HR_ORG_UNIT)
SELECT DEPARTMENT_NAME, COUNT(*) FROM COURSE_CATALOG_SUBJECT_OFFERED GROUP BY DEPARTMENT_NAME
SELECT * FROM ACADEMIC_TERMS WHERE TERM_START_DATE < CURRENT_DATE ORDER BY TERM_START_DATE DESC, SELECT * FROM ACADEMIC_TERMS WHERE TERM_START_DATE > CURRENT_DATE ORDER BY TERM_START_DATE ASC, SELECT * FROM ACADEMIC_TERMS WHERE TERM_START_DATE <= CURRENT_DATE AND TERM_END_DATE >= CURRENT_DATE ORDER BY TERM_START_DATE ASC
SELECT SE_PERSON.FULL_NAME, COUNT(DISTINCT OFFICE_LOCATION) AS LOCATION_CHANGES FROM SE_PERSON GROUP BY SE_PERSON.FULL_NAME ORDER BY LOCATION_CHANGES DESC LIMIT 1
SELECT AVG(NUM_ENROLLED_STUDENTS) AS Average_Courses FROM SUBJECT_OFFERED
SELECT MIT_STUDENT_DIRECTORY.FULL_NAME, SE_PERSON.MIT_ID FROM MIT_STUDENT_DIRECTORY I

In [44]:
df['gpt_sql_global_schema']=gpt_sql_global_schema

In [45]:
df

,question,sql,tables,gpt_sql_nocontext,gpt_sql_global_schema
0,How many organisation units in human resources...,[SELECT COUNT(*) FROM (SELECT HR_ORG_UNIT_KEY ...,"[HR_ORG_UNIT_NEW, HR_ORG_UNIT]",SELECT COUNT(*) FROM hr_organization_units WHE...,SELECT COUNT(*) FROM HR_ORG_UNIT WHERE HR_ORG_...
1,How many organisation units in human resources...,[SELECT COUNT(*) FROM (SELECT HR_ORG_UNIT_KEY ...,"[HR_ORG_UNIT_NEW, HR_ORG_UNIT]",SELECT COUNT(*) FROM human_resources WHERE is_...,SELECT COUNT(*) FROM HR_ORG_UNIT_NEW WHERE HR_...
2,COUNT the number of courses offered by each de...,"[SELECT DEPARTMENT_CODE, COUNT(*) AS COURSE_CO...","[CIS_COURSE_CATALOG, COURSE_CATALOG_SUBJECT_OF...","""SELECT department_id, COUNT(*) FROM courses G...","SELECT DEPARTMENT_NAME, COUNT(*) FROM COURSE_C..."
3,"What are the past, upcoming, and current courses?","[SELECT DISTINCT SUBJECT_TITLE, EFFECTIVE_TERM...","[CIS_COURSE_CATALOG, ACADEMIC_TERMS, DRUPAL_CO...",SELECT * FROM courses WHERE course_date < CURR...,SELECT * FROM ACADEMIC_TERMS WHERE TERM_START_...
4,Which faculty has had the most location changes?,[SELECT MAX(CNT) FROM (SELECT COUNT(*) AS CNT ...,[FCLT_BUILDING_ADDRESS_HIST],"SELECT faculty_id, COUNT(DISTINCT location_id)...","SELECT SE_PERSON.FULL_NAME, COUNT(DISTINCT OFF..."
5,What is the average number of courses a studen...,[SELECT AVG(CNT) FROM (SELECT COUNT(*) AS CNT ...,"[MIT_STUDENT_DIRECTORY, CIS_COURSE_CATALOG, MI...",SELECT AVG(course_count) FROM (SELECT student_...,SELECT AVG(NUM_ENROLLED_STUDENTS) AS Average_C...
6,Which students are employed by mit?,[SELECT DISTINCT EMPL.FULL_NAME FROM MIT_STUDE...,"[MIT_STUDENT_DIRECTORY, EMPLOYEE_DIRECTORY, MI...",SELECT student_id FROM students WHERE employme...,"SELECT MIT_STUDENT_DIRECTORY.FULL_NAME, SE_PER..."
7,What is the shortest course available this sem...,"[SELECT DISTINCT SUBJECT_TITLE, TERM_END_DATE ...","[ACADEMIC_TERMS, CIS_COURSE_CATALOG, ACADEMIC_...",SELECT MIN(course_length) FROM courses WHERE s...,"SELECT SUBJECT_ID, SUBJECT_TITLE, MIN(TERM_DUR..."
8,Show buildings and their address.,"[SELECT FB.BUILDING_NAME, FBA.* FROM FAC_BUILD...","[FAC_BUILDING, FAC_BUILDING_ADDRESS]","SELECT building_name, building_address FROM bu...","SELECT FCLT_BUILDING.BUILDING_NAME, FAC_BUILDI..."
9,How many students do we have?,[SELECT COUNT(*) FROM MIT_STUDENT_DIRECTORY;],[MIT_STUDENT_DIRECTORY],"SELECT COUNT(*) FROM students, SELECT COUNT(st...",SELECT COUNT(*) FROM MIT_STUDENT_DIRECTORY


# Data Cleaning of the gerenated answers.


In [60]:
splitted_sql = []
for i in range(len(df)):
    splitted_sql.append(df['gpt_sql_global_schema'][i].split(';'))
    if '\n' in df['gpt_sql_global_schema'][i] or '\\' in df['gpt_sql_global_schema'][i] or '"' in df['gpt_sql_global_schema'][i][0]:
        print("Pay attention to other splitting elements which GPT could have used...")
df['gpt_sql_global_schema'] = splitted_sql

# Generated for all answers of GPT a seperate ROW


In [76]:
question = []
sql = []
tables = []
gpt_sql_nocontext = []
gpt_sql_global_schema = []
for i in range(len(df)):
    for q in df['gpt_sql_global_schema'][i]:
        gpt_sql_global_schema.append(q)
        question.append(df['question'][i])
        sql.append(df['sql'][i])
        tables.append(df['tables'][i])
        gpt_sql_nocontext.append(df['gpt_sql_nocontext'][i])
data = {
    'question': question,
    'sql': sql,
    'tables': tables,
    'gpt_sql_nocontext': gpt_sql_nocontext,
    'gpt_sql_global_schema': gpt_sql_global_schema
}
df_q = pd.DataFrame(data)
df_q

,question,sql,tables,gpt_sql_nocontext,gpt_sql_global_schema
0,How many organisation units in human resources...,[SELECT COUNT(*) FROM (SELECT HR_ORG_UNIT_KEY ...,"[HR_ORG_UNIT_NEW, HR_ORG_UNIT]",SELECT COUNT(*) FROM hr_organization_units WHE...,SELECT COUNT(*) FROM HR_ORG_UNIT WHERE HR_ORG_...
1,How many organisation units in human resources...,[SELECT COUNT(*) FROM (SELECT HR_ORG_UNIT_KEY ...,"[HR_ORG_UNIT_NEW, HR_ORG_UNIT]",SELECT COUNT(*) FROM hr_organization_units WHE...,SELECT COUNT(*) FROM HR_ORG_UNIT_NEW WHERE HR_...
2,How many organisation units in human resources...,[SELECT COUNT(*) FROM (SELECT HR_ORG_UNIT_KEY ...,"[HR_ORG_UNIT_NEW, HR_ORG_UNIT]",SELECT COUNT(*) FROM human_resources WHERE is_...,SELECT COUNT(*) FROM HR_ORG_UNIT_NEW WHERE HR_...
3,COUNT the number of courses offered by each de...,"[SELECT DEPARTMENT_CODE, COUNT(*) AS COURSE_CO...","[CIS_COURSE_CATALOG, COURSE_CATALOG_SUBJECT_OF...","""SELECT department_id, COUNT(*) FROM courses G...","SELECT DEPARTMENT_NAME, COUNT(*) FROM COURSE_C..."
4,"What are the past, upcoming, and current courses?","[SELECT DISTINCT SUBJECT_TITLE, EFFECTIVE_TERM...","[CIS_COURSE_CATALOG, ACADEMIC_TERMS, DRUPAL_CO...",SELECT * FROM courses WHERE course_date < CURR...,SELECT * FROM ACADEMIC_TERMS WHERE TERM_START_...
...,...,...,...,...,...
60,Which building accomodates the most students?,"[SELECT * FROM( SELECT DISTINCT BUILDING_NAME,...","[FCLT_ROOMS, MIT_STUDENT_DIRECTORY, FCLT_BUILD...","SELECT building_name, COUNT(student_id) AS stu...","SELECT FCLT_BUILDING.BUILDING_NAME, MAX(FCLT_R..."
61,Which building has on average the most rooms p...,"[SELECT DISTINCT BUILDING_NAME, NUM_ROOMS/NUM_...","[FCLT_ROOMS, FCLT_BUILDING, FAC_ROOMS, FAC_BUI...","SELECT building_name, AVG(num_rooms) as avg_ro...","SELECT FCLT_BUILDING.BUILDING_NAME, AVG(FAC_FL..."
62,What is the most common usage for all rooms.,"[SELECT * FROM(SELECT DISTINCT MAJOR_USE, NUM_...","[FCLT_ROOMS, FCLT_MAJOR_USE, FAC_ROOMS, FAC_MA...","SELECT room_usage, COUNT(*) AS usage_count FRO...","SELECT MAJOR_USE_DESC, COUNT(*) AS COUNT FROM ..."
63,Which IAP instructor has held the most session...,"[SELECT * FROM (SELECT DISTINCT PERSON_NAME, A...","[IAP_SUBJECT_DETAIL, IAP_SUBJECT_PERSON]","SELECT instructor_name, course_id, COUNT(*) as...","SELECT IAP_SUBJECT_PERSON.PERSON_NAME, COUNT(I..."


# Detect all relevant tables GPT used.
Starting with the tables used by the GPT without any context.

In [88]:
tables = []
cnt = 0 
for q in df_q['gpt_sql_nocontext']:
    print(q)
    cnt = cnt + 1
    if '"' in q:
        q = q[1:-1]
    try: 
        t = Parser(q).tables
        tables.append([v.upper() for v in t])
    except:
        tables.append(None)
df_q['tables_referred_by_gpt_sql_nocontext'] = tables

SELECT COUNT(*) FROM hr_organization_units WHERE relabeled = 'Yes', SELECT COUNT(*) FROM hr_organization_units WHERE relabeled = true
SELECT COUNT(*) FROM hr_organization_units WHERE relabeled = 'Yes', SELECT COUNT(*) FROM hr_organization_units WHERE relabeled = true
SELECT COUNT(*) FROM human_resources WHERE is_new = 'Yes' AND is_relabelled = 'No', SELECT COUNT(*) FROM organisation_units WHERE status = 'New' AND relabelled = 'No'
"SELECT department_id, COUNT(*) FROM courses GROUP BY department_id", "SELECT department_name, COUNT(course_id) FROM departments INNER JOIN courses ON departments.department_id = courses.department_id GROUP BY department_name"
SELECT * FROM courses WHERE course_date < CURRENT_DATE, SELECT * FROM courses WHERE course_date > CURRENT_DATE, SELECT * FROM courses WHERE course_date = CURRENT_DATE
SELECT * FROM courses WHERE course_date < CURRENT_DATE, SELECT * FROM courses WHERE course_date > CURRENT_DATE, SELECT * FROM courses WHERE course_date = CURRENT_DATE
SELE

And then detecting the tables used by the GPT with the table-schemas.


In [86]:
tables = []
cnt = 0 
for q in df_q['gpt_sql_global_schema']:
    print(q)
    cnt = cnt + 1
    try: 
        t = Parser(q).tables
        tables.append([v.upper() for v in t])
    except:
        tables.append(None)
df_q['tables_referred_by_gpt_sql_global_schema'] = tables


SELECT COUNT(*) FROM HR_ORG_UNIT WHERE HR_ORG_UNIT_TITLE != HR_ORG_UNIT_ID
SELECT COUNT(*) FROM HR_ORG_UNIT_NEW WHERE HR_ORG_UNIT_TITLE != HR_ORG_UNIT_ID
SELECT COUNT(*) FROM HR_ORG_UNIT_NEW WHERE HR_ORG_UNIT_ID NOT IN (SELECT HR_ORG_UNIT_ID FROM HR_ORG_UNIT)
SELECT DEPARTMENT_NAME, COUNT(*) FROM COURSE_CATALOG_SUBJECT_OFFERED GROUP BY DEPARTMENT_NAME
SELECT * FROM ACADEMIC_TERMS WHERE TERM_START_DATE < CURRENT_DATE ORDER BY TERM_START_DATE DESC
SELECT * FROM ACADEMIC_TERMS WHERE TERM_START_DATE > CURRENT_DATE ORDER BY TERM_START_DATE ASC
SELECT * FROM ACADEMIC_TERMS WHERE TERM_START_DATE <= CURRENT_DATE AND TERM_END_DATE >= CURRENT_DATE ORDER BY TERM_START_DATE ASC
SELECT SE_PERSON.FULL_NAME, COUNT(DISTINCT OFFICE_LOCATION) AS LOCATION_CHANGES FROM SE_PERSON GROUP BY SE_PERSON.FULL_NAME ORDER BY LOCATION_CHANGES DESC LIMIT 1
SELECT AVG(NUM_ENROLLED_STUDENTS) AS Average_Courses FROM SUBJECT_OFFERED
SELECT MIT_STUDENT_DIRECTORY.FULL_NAME, SE_PERSON.MIT_ID FROM MIT_STUDENT_DIRECTORY INNE

# Save Dataframe with generated answers


In [240]:
df_q.to_csv('/home/skikk/Dokumente/uni/WS2324/MIT/llm-enterprise/data/gpt_sql_v1.5.csv', index=False)
df_q

question  \
0                                           How many organisation units in human resources are relabeled?   
1                                           How many organisation units in human resources are relabeled?   
2                               How many organisation units in human resources are new and not relabeled?   
3                                                 COUNT the number of courses offered by each department.   
4                                                       What are the past, upcoming, and current courses?   
5                                                       What are the past, upcoming, and current courses?   
6                                                       What are the past, upcoming, and current courses?   
7                                                        Which faculty has had the most location changes?   
8                                          What is the average number of courses a student can enroll in?   
9                                                                     Which students are employed by mit?   
10                                                   What is the shortest course available this semester?   
11                                                                      Show buildings and their address.   
12                                                                          How many students do we have?   
13                                                                       Show all statuses for materials?   
14                                                       What are the different cities WHERE people live?   
15                                                       What are the different cities WHERE people live?   
16                       Give the average number of courses in one department with more than 100 courses.   
17                                                               Count the number of rooms in a building.   
18                                                          Count the number of space-units per building.   
19                 For courses that have descriptions of Regular and Initial, what are their start dates?   
20                     What are the names of the 3 departments which offer the most courses per semester?   
21                     What are the names of the 3 departments which offer the most courses per semester?   
22                     What are the names of the 3 departments which offer the most courses per semester?   
23                                                           WHERE is Prof. Michael Stonebraker's office?   
24                                                                Which employees at MIT have a namesake?   
25                         Show the name for subjects which are enrollable for Computer Science Students.   
26                                                           How many students does each department have?   
27                                          What categories are there for independent activities periods?   
28                                                                  Find the city with the post code 501.   
29                                                               How many employees are there all in all?   
30                                                            How many departments does each school have?   
31               Show the name, kerberos credentials, and email for the employee with the name 'Michael'.   
32               Show the name, kerberos credentials, and email for the employee with the name 'Michael'.   
33               Show the name, kerberos credentials, and email for the employee with the name 'Michael'.   
34                                                Show the school with the largest number of departments.   
35                                                 What are the courses with the maximum number of units?   
36                                                 What are the 

# READ DATAFRAME


In [ ]:
df_q = pd.read_csv('/home/skikk/Dokumente/uni/WS2324/MIT/llm-enterprise/data/gpt_sql_v1.6.csv')

In [5]:
sql = []
tables = []
tables_referred_by_gpt_sql_global_schema = []
tables_referred_by_gpt_sql_nocontext = []
sql_outputsizes = []
for i in range(len(df_q)):
    sql.append(ast.literal_eval(df_q['sql'][i]))
    tables.append(ast.literal_eval(df_q['tables'][i]))
    tables_referred_by_gpt_sql_global_schema.append(ast.literal_eval(df_q['tables_referred_by_gpt_sql_global_schema'][i]))
    tables_referred_by_gpt_sql_nocontext.append(ast.literal_eval(df_q['tables_referred_by_gpt_sql_nocontext'][i]))
    sql_outputsizes.append(ast.literal_eval(df_q['sql_outputsizes'][i]))
df_q['sql'] = sql
df_q['tables'] = tables
df_q['tables_referred_by_gpt_sql_global_schema'] = tables_referred_by_gpt_sql_global_schema
df_q['tables_referred_by_gpt_sql_nocontext'] = tables_referred_by_gpt_sql_nocontext
df_q['sql_outputsizes'] = sql_outputsizes

In [95]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
df_q

question  \
0                                           How many organisation units in human resources are relabeled?   
1                                           How many organisation units in human resources are relabeled?   
2                               How many organisation units in human resources are new and not relabeled?   
3                                                 COUNT the number of courses offered by each department.   
4                                                       What are the past, upcoming, and current courses?   
5                                                       What are the past, upcoming, and current courses?   
6                                                       What are the past, upcoming, and current courses?   
7                                                        Which faculty has had the most location changes?   
8                                          What is the average number of courses a student can enroll in?   
9                                                                     Which students are employed by mit?   
10                                                   What is the shortest course available this semester?   
11                                                                      Show buildings and their address.   
12                                                                          How many students do we have?   
13                                                                       Show all statuses for materials?   
14                                                       What are the different cities WHERE people live?   
15                                                       What are the different cities WHERE people live?   
16                       Give the average number of courses in one department with more than 100 courses.   
17                                                               Count the number of rooms in a building.   
18                                                          Count the number of space-units per building.   
19                 For courses that have descriptions of Regular and Initial, what are their start dates?   
20                     What are the names of the 3 departments which offer the most courses per semester?   
21                     What are the names of the 3 departments which offer the most courses per semester?   
22                     What are the names of the 3 departments which offer the most courses per semester?   
23                                                           WHERE is Prof. Michael Stonebraker's office?   
24                                                                Which employees at MIT have a namesake?   
25                         Show the name for subjects which are enrollable for Computer Science Students.   
26                                                           How many students does each department have?   
27                                          What categories are there for independent activities periods?   
28                                                                  Find the city with the post code 501.   
29                                                               How many employees are there all in all?   
30                                                            How many departments does each school have?   
31               Show the name, kerberos credentials, and email for the employee with the name 'Michael'.   
32               Show the name, kerberos credentials, and email for the employee with the name 'Michael'.   
33               Show the name, kerberos credentials, and email for the employee with the name 'Michael'.   
34                                                Show the school with the largest number of departments.   
35                                                 What are the courses with the maximum number of units?   
36                                                 What are the 

# Calculate the output-size


In [132]:
def calculate_output_size(query, tables=[]):

    if 'LIMIT' in query:
        contains_limit = True
        limit_number = re.search(r'\bLIMIT\s+(\d+)', query)
        limit_value = int(limit_number.group(1))
        query = query[:query.index('LIMIT ')]
        query = 'SELECT * FROM (' + query + ') WHERE ROWNUM <=' + str(limit_value)
    formatted_query = f"SELECT DISTINCT COUNT(*) FROM({query})"
    rows = execute_query(formatted_query, tables)
    return rows[0][0]

### Calculate outputs for expected and actual results


In [ ]:
sizes = []
for q in df_q['gpt_sql_global_schema']:
    sizes.append(calculate_output_size(q))
    print(calculate_output_size(q))
df_q['gpt_sql_global_schema_outputsizes'] = sizes
sizes = []
cnt = 0
for i in range(len(df_q)):
    size = []
    for j in range(len(df_q['sql'][i])):
        print(df_q['sql'][i][j][:-1])
        output = calculate_output_size(df_q['sql'][i][j][:-1], df_q['tables'][i])
        size.append(output)
        print(output)
    sizes.append(size)
df_q['sql_outputsizes'] = sizes

SELECT COUNT(*) FROM (SELECT HR_ORG_UNIT_KEY FROM HR_ORG_UNIT_NEW INTERSECT SELECT HR_ORG_UNIT_KEY FROM HR_ORG_UNIT);
1
1
SELECT COUNT(*) FROM (SELECT HR_ORG_UNIT_KEY FROM HR_ORG_UNIT_NEW INTERSECT SELECT HR_ORG_UNIT_KEY FROM HR_ORG_UNIT);
2
1
SELECT COUNT(*) FROM (SELECT HR_ORG_UNIT_KEY FROM HR_ORG_UNIT_NEW INTERSECT SELECT HR_ORG_UNIT_KEY FROM HR_ORG_UNIT);
3
1
SELECT COUNT(*) FROM (SELECT HR_ORG_UNIT_KEY FROM HR_ORG_UNIT_NEW INTERSECT SELECT HR_ORG_UNIT_KEY FROM HR_ORG_UNIT);
4
62
SELECT COUNT(*) FROM (SELECT HR_ORG_UNIT_KEY FROM HR_ORG_UNIT_NEW INTERSECT SELECT HR_ORG_UNIT_KEY FROM HR_ORG_UNIT);
4
62
SELECT COUNT(*) FROM (SELECT HR_ORG_UNIT_KEY FROM HR_ORG_UNIT_NEW INTERSECT SELECT HR_ORG_UNIT_KEY FROM HR_ORG_UNIT);
4
58
SELECT COUNT(*) FROM (SELECT HR_ORG_UNIT_KEY FROM HR_ORG_UNIT_NEW INTERSECT SELECT HR_ORG_UNIT_KEY FROM HR_ORG_UNIT);
4
71
SELECT COUNT(*) FROM (SELECT HR_ORG_UNIT_KEY FROM HR_ORG_UNIT_NEW INTERSECT SELECT HR_ORG_UNIT_KEY FROM HR_ORG_UNIT);
4
0
SELECT COUNT(*) FROM

### Check how many sizes are matching and how many queries are failing


In [9]:
cnt_true = 0
cnt_neg = 0
for i in range(len(df_q)):
    if df_q['gpt_sql_global_schema_outputsizes'][i] in df_q['sql_outputsizes'][i] and df_q['gpt_sql_global_schema_outputsizes'][i]>0:
        cnt_true += 1;
    if df_q['gpt_sql_global_schema_outputsizes'][i] < 0:
        cnt_neg +=1
print(cnt_true)
print(len(df_q))
print(cnt_neg)

21
65
17


# Compare Results
Idea to base the result comparison based on the output size:
<ol>
  <li>We check at first only outputs with the equal amount of rows in their result.</li>
  <li>outputs of size 10 or less are completely computable and every element is comparable (without sorting).</li>
  <li>larger outputs are compared via sorting/hashing and comparison.</li>
  <li>Failing queries should be marked as incorrect attached with their error message.</li>
  <li>(Maybe later for fuzzy comparisons)Manual checks: queries with no output, queries with different dimensions, and duplications.</li>
</ol>

1. We check outputsize and dimensionality.

In [ ]:
correct_size = []
for i in range(len(df_q)):
    if(df_q['gpt_sql_global_schema_outputsizes'][i] not in df_q['sql_outputsizes'][i]):
        correct_size.append(0)
    else:
        correct_size.append(-1)
correct_size

In [131]:
def execute_query(query, tables):
    connection = cx_Oracle.connect(user=username, password=password, dsn=dsn)
    
    if 'LIMIT' in query:
        contains_limit = True
        limit_number = re.search(r'\bLIMIT\s+(\d+)', query)
        limit_value = int(limit_number.group(1))
        query = query[:query.index('LIMIT ')]
        query = 'SELECT * FROM (' + query + ') WHERE ROWNUM <=' + str(limit_value)

    # Set input type handler to cx_Oracle.STRING
    connection.inputtypehandler = cx_Oracle.STRING
    # Create a cursor object
    cursor = connection.cursor()
    
    # Set DEFINE OFF
    cursor.setoutputsize(0)
    
    # Execute SQL queries
    try:
        cursor.execute(query)
        # Fetch the results
        rows = cursor.fetchall()
    except Exception as e1:
        #Preprocessing wueries
        for table in tables:
            table_mod = ' WAREUSER.' + table
            query = query.replace(' ' + table, table_mod)
            print(query)
        try:
            cursor.execute(query)
            # Fetch the results
            rows = cursor.fetchall()
        except Exception as e2:
            # Print the error message to stderr
            print("An error occurred:", str(e1))
            print("An error occurred:", str(e2))
            return [[-1]]
    cursor.close()
    connection.close()
    return rows

### Start with single comparisons
Generate output and compare result

In [144]:
def compare_single_outputs(expected_queries, expected_tables, given_query, given_tables):
    expected_results = []
    for query in expected_queries:
        expected_result = execute_query(query[:-1], expected_tables)[0][0]
        expected_results.append(expected_result)
        print(expected_result)
    actual_result = execute_query(given_query, given_tables)[0][0]
    print(actual_result)
    return actual_result in expected_results

In [146]:
correct = []
cnt=0
for i in range(len(df_q)):
    if (df_q['gpt_sql_global_schema_outputsizes'][i] == 1) and (1 in df_q['sql_outputsizes'][i]) and df_q['correct'][i]<0:
        print(i)
        print(df_q['sql'][i])
        print(df_q['gpt_sql_global_schema'][i])
        res = compare_single_outputs(df_q['sql'][i], df_q['tables'][i], df_q['gpt_sql_global_schema'][i], df_q['tables_referred_by_gpt_sql_global_schema'][i])
        if res:
            cnt +=1
        correct.append(int(res))
        print(res)
    else:
        correct.append(df_q['correct'][i])
print(cnt)

0
['SELECT COUNT(*) FROM (SELECT HR_ORG_UNIT_KEY FROM HR_ORG_UNIT_NEW INTERSECT SELECT HR_ORG_UNIT_KEY FROM HR_ORG_UNIT);']
SELECT COUNT(*) FROM HR_ORG_UNIT WHERE HR_ORG_UNIT_TITLE != HR_ORG_UNIT_ID
635
635
True
1
['SELECT COUNT(*) FROM (SELECT HR_ORG_UNIT_KEY FROM HR_ORG_UNIT_NEW INTERSECT SELECT HR_ORG_UNIT_KEY FROM HR_ORG_UNIT);']
SELECT COUNT(*) FROM HR_ORG_UNIT_NEW WHERE HR_ORG_UNIT_TITLE != HR_ORG_UNIT_ID
635
685
False
2
['SELECT COUNT(*) FROM (SELECT HR_ORG_UNIT_KEY FROM HR_ORG_UNIT_NEW MINUS SELECT HR_ORG_UNIT_KEY FROM HR_ORG_UNIT);']
SELECT COUNT(*) FROM HR_ORG_UNIT_NEW WHERE HR_ORG_UNIT_ID NOT IN (SELECT HR_ORG_UNIT_ID FROM HR_ORG_UNIT)
50
50
True
7
['SELECT MAX(CNT) FROM (SELECT COUNT(*) AS CNT FROM FCLT_BUILDING_ADDRESS_HIST GROUP BY FCLT_BUILDING_KEY);']
SELECT SE_PERSON.FULL_NAME, COUNT(DISTINCT OFFICE_LOCATION) AS LOCATION_CHANGES FROM SE_PERSON GROUP BY SE_PERSON.FULL_NAME ORDER BY LOCATION_CHANGES DESC LIMIT 1
SELECT MAX(CNT) FROM (SELECT COUNT(*) AS CNT FROM WAREUSER.

In [156]:
print(len(correct_size))
print(sum(correct_size))
df_q['gpt_sql_global_schema_outputsizes'].value_counts().get(-1, 0)

65
-25


16

### Less than 10 results
Idea: check if the sizes match and whether each result is contain in one another.

In [181]:
def compare_countable_outputs(expected_queries, expected_tables, given_query, given_tables):
    expected_results = []
    actual_result = execute_query(given_query, given_tables)
    for query in expected_queries:
        expected_result = execute_query(query[:-1], expected_tables)
        if len(expected_result)==0 and len(expected_result)==0:
            return True
        expected_results.append(expected_result)
    is_matching = []
    for i in range(len(expected_results)):
        match = True
        for j in range(len(expected_results[i])):
            if not expected_results[i][j] in actual_result:
                match= False
                break
        is_matching.append(match)
    return any(is_matching)

In [ ]:
correct_m = []
cnt=0
for i in range(len(df_q)):
    if (df_q['gpt_sql_global_schema_outputsizes'][i] >=0) and (df_q['gpt_sql_global_schema_outputsizes'][i] <=10) and (df_q['correct'][i]==-1):
        res = compare_countable_outputs(df_q['sql'][i], df_q['tables'][i], df_q['gpt_sql_global_schema'][i], df_q['tables_referred_by_gpt_sql_global_schema'][i])
        if res:
            cnt +=1
        correct_m.append(int(res))
    else:
        correct_m.append(df_q['correct'][i])
print(cnt)
correct_m

In [185]:
df_q['correct']=correct_m

### Larger Output sizes
Idea: check if samples from the expected result are all included in the actual result.

In [188]:
def compare_incountable_outputs(expected_comparable_query, expected_tables, given_query, given_tables, dict_res):
    expected_result = execute_query(expected_comparable_query[:-1], expected_tables)
    actual_result = execute_query(given_query, given_tables)
    if len(expected_result[0]) != len(actual_result[0]):
        return -1
    
    sorted_expected_data = sorted(expected_result, key=lambda x: x[0])
    dict_res[expected_comparable_query]=sorted_expected_data
    sorted_actual_data = sorted(actual_result, key=lambda x: x[0])
    for i in range(len(sorted_expected_data)):
        if sorted_expected_data[i] != sorted_actual_data[i]:
            return False
    return True

In [ ]:
def compare_incountable_with_precalculated(sorted_expected_data, given_query, given_tables):
    sorted_actual_data = sorted(actual_result, key=lambda x: x[0])
    for i in range(len(sorted_expected_data)):
        if sorted_expected_data[i] != sorted_actual_data[i]:
            return False
    return True

In [ ]:
correct_l = []
dict_res={}
cnt=0
for i in range(len(df_q)):
    if (df_q['gpt_sql_global_schema_outputsizes'][i] >10) and (df_q['correct'][i]==-1):
        res = []
        for j in range(len(df_q['sql'][i])):
            if df_q['sql_outputsizes'][i][j] == df_q['gpt_sql_global_schema_outputsizes'][i]:
                if df_q['sql'][i][j] in dict_res.keys():
                    res.append(compare_incountable_with_precalculated(dict_res[df_q['sql'][i][j], df_q['gpt_sql_global_schema'][i], df_q['tables_referred_by_gpt_sql_global_schema'][i]))
                else:
                    res.append(compare_incountable_outputs(df_q['sql'][i][j], df_q['tables'][i], df_q['gpt_sql_global_schema'][i], df_q['tables_referred_by_gpt_sql_global_schema'][i]))
        if any(res):
            cnt +=1
        correct_l.append(int(any(res)))
    else:
        correct_l.append(df_q['correct'][i])
print(cnt)
correct_l

In [190]:
df_q['correct'] = correct_l

In [235]:
print("Total size of the data:   " + str(len(df_q)))
wrong_out = len(df_q)+sum(correct_size)
print("#wrong outputsize:        " + str(wrong_out))
print("   #wrong sqls:           " + str(df_q['gpt_sql_global_schema_outputsizes'].value_counts().get(-1, 0)))
print("#correct outputsize:      " + str(-1*sum(correct_size)))
print("   #1size-oputput:        " + str(sum((e==0 or e ==1) for e in correct)-wrong_out))
print("      #correct 1-size:    " + str(sum(e ==1 for e in correct)))
print("   #>1size-oputput:       " + str(-sum(correct_size)+wrong_out - sum((e==0 or e ==1) for e in correct)))
print("      #<11size-oputput:   " + str(sum(e==0 or e ==1 for e in correct_m)-sum(e==0 or e ==1 for e in correct)))
print("         #0size-oputput:  " + str(1))
print("         #corr<10size-out:" + str(sum(e ==1 for e in correct_m)-sum(e ==1 for e in correct)))
print("      #>10size-oputput:   " + str(sum(e==0 or e ==1 for e in correct_l)- sum(e==0 or e ==1 for e in correct_m)))
print("         #corr>10size-out:" + str(sum(e ==1 for e in correct_l)- sum(e ==1 for e in correct_m)))
print("Total #correct:           " + str(sum(df_q['correct'])))

Total size of the data:   65
#wrong outputsize:        40
   #wrong sqls:           16
#correct outputsize:      25
   #1size-oputput:        17
      #correct 1-size:    8
   #>1size-oputput:       8
      #<11size-oputput:   5
         #0size-oputput:  1
         #corr<10size-out:5
      #>10size-oputput:   3
         #corr>10size-out:3
Total #correct:           16


# Calculate Accuracy, Precision, Recall and F1-Score

Accuracy: $\frac{\text{Number of correct predictions}}{\text{Total number of predictions}}$

Precision: $\frac{\text{True Positives}}{\text{True Positives} + \text{False Positives}}$

Recall: $\frac{\text{True Positives}}{\text{True Positives} + \text{False Negatives}}$

F1-Score: $2 \times \frac{\text{Precision} \times \text{Recall}}{\text{Precision} + \text{Recall}}$

In [216]:
print('Accuracy: ' + str(sum(df_q['correct'])/len(df_q) * 100) + '%')


Accuracy: 24.615384615384617%


## Based on Table usage

In [325]:
accuracy = []
for t0, t1 in zip(df['tables_referred'], df['gpt_sql_global_schema_retrieved']):
    a = None 
    k='WAREUSER'
    f = False
    for t in t0:
        # ignore the sql queries referring to the tables with the 'WAREUSER' 
        # prefix, for which we didn't find the schemas in the schema directory 
        if t.find(k) > -1: 
            f = True
            break
    if t0 and t1 and not f: 
        a = len(set(t0).intersection(set(t1)))/len(t0)
    accuracy.append(a)
df['gpt_sql_global_schema_accuracy']=accuracy 

In [ ]:
accuracy = []
for t0, t1 in zip(df_q['tables_referred'], df_q['gpt_sql_global_schema_retrieved']):
    a = None 
    if t0 and t1 and not f: 
        a = len(set(t0).intersection(set(t1)))/len(t0)
    accuracy.append(a)
df_q['gpt_sql_global_schema_accuracy']=accuracy 